# Getting Started with Fine-Tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/smartrics/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/smartrics/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [ ]:
import os
import subprocess

repo_dir = "/content/mistral-finetune"
repo_url = "https://github.com/smartrics/mistral-finetune.git"

if os.path.isdir(repo_dir):
    print("Directory 'mistral-finetune' exists. Pulling latest changes...")
    subprocess.run(["git", "-C", repo_dir, "pull"], check=True)
else:
    print("Directory 'mistral-finetune' does not exist. Cloning repository...")
    subprocess.run(["git", "clone", repo_url, repo_dir], check=True)
print("finished!")

Install all required dependencies:

In [ ]:
!pip install -r /content/mistral-finetune/requirements.txt

## Model download

In [ ]:
!pip install huggingface_hub

In [14]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

# Define the target path directly in /content/mistral_models
mistral_models_path = Path("/content/mistral_models/7B-Instruct-v0.3")
mistral_models_path.mkdir(parents=True, exist_ok=True)

# Download directly to the target directory
snapshot_download(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"],
    local_dir=mistral_models_path
)

print(f"Model downloaded to {mistral_models_path}")


## Dataset

Use the data in `/content/mistral-finetune/data`

In [16]:
!ls /content/mistral-finetune/data

prepare.py  test_data.jsonl  training_data.jsonl  validation_data.jsonl


In [17]:
# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

/content/mistral-finetune


In [ ]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


## Start training

In [19]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [20]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
data:
  instruct_data: "data/test_data.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "data/validation_data.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models/7B-Instruct-v0.3"  # Change to downloaded path
lora:
  rank: 64

# optim
seq_len: 8192
batch_size: 1
max_steps: 300
optim:
  lr: 6.e-5
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/mistral_models/7B-Instruct-v0.3_trained"  # Fill

wandb:
  project: None # your wandb project name
  run_name: "" # your wandb run name
  key: "" # your wandb api key
  offline: True

"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [21]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

import os
os.environ["WANDB_MODE"] = "disabled"


In [ ]:
import os
import zipfile

# Define paths
zip_path = "/content/mistral_models/7B-Instruct-v0.3_trained.zip"
extract_path = "/content/mistral_models"

if os.path.exists(zip_path):
    print("Zip file found. Extracting...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
else:
    # start training
    print("Zip file not found. Removing existing model directory and starting training...")
    !rm -rf /content/mistral_models/7B-Instruct-v0.3_trained
    !torchrun -m train example.yaml
print("finished!")

In [ ]:
# building llama.cpp
%cd /content
!git clone https://github.com/ggerganov/llama.cpp.git
!mkdir -p llama.cpp/build
%cd llama.cpp/build
!cmake .. && make -j$(nproc)

In [ ]:
%cd /content/llama.cpp
!pip install -r requirements.txt


In [12]:
# Merge trained weights with the Base Model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from safetensors.torch import load_file

# Define paths
BASE_MODEL_PATH = "/content/mistral_models/7B-Instruct-v0.3"
LORA_ADAPTER_PATH = "/content/mistral_models/7B-Instruct-v0.3_trained/checkpoints/checkpoint_000300/consolidated/lora.safetensors"
OUTPUT_MODEL_PATH = "/content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_PATH, torch_dtype=torch.float16)

# Load LoRA weights from safetensors
lora_weights = load_file(LORA_ADAPTER_PATH)

# Manually update base model's state dict with LoRA weights
base_model.load_state_dict(lora_weights, strict=False)

# Save the fully merged model
base_model.save_pretrained(OUTPUT_MODEL_PATH)

# Save tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH)
tokenizer.save_pretrained(OUTPUT_MODEL_PATH)

print(f"✅ Model successfully merged and saved to: {OUTPUT_MODEL_PATH}")



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

✅ Model successfully merged and saved to: /content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind


## Convert to Ollama
Ollama requires models in GGUF format, so we’ll use llama.cpp to convert your Mistral-7B-Instruct-v0.3-tamarind model

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

# Define paths
MODEL_PATH = "/content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind"
GGUF_OUTPUT_PATH = "/content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind.gguf"

# Load the model
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Export to GGUF
model.save_pretrained(GGUF_OUTPUT_PATH)
tokenizer.save_pretrained(GGUF_OUTPUT_PATH)

print(f"✅ Model converted to GGUF format: {GGUF_OUTPUT_PATH}")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model converted to GGUF format: /content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind.gguf


In [ ]:
!zip -r /content/Mistral-7B-Instruct-v0.3-tamarind.zip /content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind.gguf


  adding: content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind.gguf/ (stored 0%)
  adding: content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind.gguf/model-00001-of-00003.safetensors